In [3]:
from bs4 import BeautifulSoup
import requests as req
from time import sleep
import pandas as pd
#from tqdm import tqdm


# configure pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [100]:
page = req.get("https://www.spotrac.com/nfl/chicago-bears/contracts/").text
soup = BeautifulSoup(page, "html.parser")
data = []
selects = soup.find('select', attrs={"class":"team-select"})
option_list = selects.find_all("option")
team_names = [o['value'] for o in option_list if o['value'] != "all"]
team_names

['arizona-cardinals',
 'atlanta-falcons',
 'baltimore-ravens',
 'buffalo-bills',
 'carolina-panthers',
 'chicago-bears',
 'cincinnati-bengals',
 'cleveland-browns',
 'dallas-cowboys',
 'denver-broncos',
 'detroit-lions',
 'green-bay-packers',
 'houston-texans',
 'indianapolis-colts',
 'jacksonville-jaguars',
 'kansas-city-chiefs',
 'las-vegas-raiders',
 'los-angeles-chargers',
 'los-angeles-rams',
 'miami-dolphins',
 'minnesota-vikings',
 'new-england-patriots',
 'new-orleans-saints',
 'new-york-giants',
 'new-york-jets',
 'philadelphia-eagles',
 'pittsburgh-steelers',
 'san-francisco-49ers',
 'seattle-seahawks',
 'tampa-bay-buccaneers',
 'tennessee-titans',
 'washington-commanders']

In [101]:
pd_list = []
for team in team_names:
    print(f"Processing {team}...")
    url = f"https://www.spotrac.com/nfl/{team}/contracts/"
    page = req.get(url).text
    soup = BeautifulSoup(page, "html.parser")
    data = []
    table = soup.find('table')
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        tds = row.find_all("td")
        player_name = tds[0].find_all("a")[0].text.strip()
        position = tds[1].text.strip()
        age = tds[2].text.strip()
        exp = tds[3].text.strip()
        contract_amount = tds[4].find_all("span")[0].text.strip()
        contract_length = tds[4].find_all("span")[1].text.strip().split(" ")[0]
        guaranteed_amount = tds[6].text.strip()
        contract_expires = tds[7].text.strip()
        acquired = tds[8].text.strip()
        data.append([player_name, position, age, exp, contract_amount, contract_length, guaranteed_amount, contract_expires, acquired]) # Get rid of empty values
    columns_names = table.find_all("thead")
    for row in columns_names:
        cols = [d.text.strip() for d in row.find_all("th")]
    sleep(10)
    df = pd.DataFrame(data=data, columns=['player_name','position','age','exp','contract_amount',
                                          'contract_length','guaranteed_amount','contract_expires','acquired'])
    df["tm"] = team
    print(df.head())
    pd_list.append(df)

Processing arizona-cardinals...
       player_name position age exp contract_amount contract_length guaranteed_amount contract_expires    acquired                 tm
0     Kyler Murray       QB  25   4       230500000               5      $160,000,000             2029     Drafted  arizona-cardinals
1      Budda Baker        S  26   6        59000000               4       $33,100,000             2025     Drafted  arizona-cardinals
2  DeAndre Hopkins       WR  30   9        54500000               2       $60,050,000             2025      Traded  arizona-cardinals
3   D.J. Humphries       LT  28   8        51763530               3       $32,823,529             2026     Drafted  arizona-cardinals
4      Justin Pugh        G  32  10        44775000               5       $15,750,000             2023  Free Agent  arizona-cardinals
Processing atlanta-falcons...
     player_name position age exp contract_amount contract_length guaranteed_amount contract_expires    acquired               tm
0   

In [102]:
df = pd.concat(pd_list)

In [104]:
df.to_csv("data/all_2021_nfl_contracts.csv")